<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Ensemble_with_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ensemble


In [25]:
'''imports '''
from google.colab import files
uploaded = files.upload()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from tensorflow.keras.models import Model


from tqdm.auto import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# The complete datasets needed
train_df = pd.read_json('train_df_complete.json', orient="records", lines=True)
dev_df = pd.read_json('dev_df_complete.json', orient="records", lines=True)


# The logits from the base model based on the descriptions, on the training and dev set, available in the datasets folder
# on github
logits_train_model_description = np.load("train_logits_description.npy")
logits_train_model_features = np.load("train_logits.npy")

# The logits from the base mode based on the extracted features, on the training and dev set , available in datasets folder
# on the github
logits_dev_model_descriptions = np.load("dev_logits_description.npy")
logits_dev_model_features = np.load("dev_logits.npy")

Saving dev_logits_description.npy to dev_logits_description.npy
Saving train_logits_description.npy to train_logits_description (1).npy


### The dataset preparation for the ensemble


In [26]:
df_train_description = pd.DataFrame(logits_train_model_description, columns=['logit_0', 'logit_1', 'logit_2'])
df_train_description['label'] = train_df['label_int']
dev_df_description = pd.DataFrame(logits_dev_model_descriptions, columns=['logit_0', 'logit_1', 'logit_2'])
dev_df_description['label'] = dev_df['label_int']


In [27]:
'''sanity check on the dataframe'''
df_train_description.head(10)

,logit_0,logit_1,logit_2,label
0,1.747782,-0.548266,-2.989935,0
1,1.208469,3.105253,-1.913423,1
2,1.216628,4.417734,-1.742478,1
3,1.107998,0.357187,-3.111727,1
4,1.117797,4.252902,-1.600649,1
5,1.059856,3.021012,-1.723123,1
6,2.096429,-0.963109,-2.976534,0
7,1.200607,0.291133,-3.217273,1
8,1.124358,4.485882,-1.421949,1
9,1.671226,-0.354426,-3.153117,0


## The ensemble nn working on the logits of the trained two base nns

here we try to combine two classifiers, one basing on the neural network working on features extracted frrom the wikidata/wikipedia, and the second one basing on the neural network working on the embedding representation of the description of the wikipedia/wikidata entities. We combine them with a meta lerner that as training set  takes the logits for each label from both of the trained base neural networks, achieved on the train set. And the validation are the logits of these two networks acquired by predcition on the validation set.

In [28]:


early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

train_labels = df_train_description['label']
dev_labels = dev_df_description['label']

train_labels_nn = to_categorical(train_labels, num_classes=3)
dev_labels_nn = to_categorical(dev_labels, num_classes=3)

train_vectors = np.concatenate((logits_train_model_description, logits_train_model_features), axis=1)
dev_vectors = np.concatenate((logits_dev_model_descriptions, logits_dev_model_features), axis=1)




model_Ensemble = models.Sequential()


input_layer = layers.Input(shape=(6,), name='input_layer')
model_Ensemble.add(input_layer)

for units in [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 32, 32]:
    dense_layer = layers.Dense(units, activation='relu')
    relu_layer = LeakyReLU(alpha=0.1)
    batch_norm_layer = layers.BatchNormalization()
    dropout_layer = layers.Dropout(0.3 if units > 128 else 0.2)
    model_Ensemble.add(dense_layer)
    model_Ensemble.add(relu_layer)
    model_Ensemble.add(batch_norm_layer)
    model_Ensemble.add(dropout_layer)

model_Ensemble.add(layers.Dense(3, activation='softmax'))


model_Ensemble.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])




history = model_Ensemble.fit(
    train_vectors, train_labels_nn,
    validation_data=(dev_vectors, dev_labels_nn),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr]
)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 22s 80ms/step - accuracy: 0.4894 - loss: 1.1634 - val_accuracy: 0.6600 - val_loss: 0.7283 - learning_rate: 0.0010
Epoch 2/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.8677 - loss: 0.3896 - val_accuracy: 0.6467 - val_loss: 0.7884 - learning_rate: 0.0010
Epoch 3/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.9022 - loss: 0.3030 - val_accuracy: 0.7400 - val_loss: 0.6819 - learning_rate: 0.0010
Epoch 4/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9025 - loss: 0.2976 - val_accuracy: 0.7367 - val_loss: 0.6873 - learning_rate: 0.0010
Epoch 5/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 7s 73ms/step - accuracy: 0.9060 - loss: 0.2789 - val_accuracy: 0.7167 - val_loss: 0.7324 - learning_rate: 0.0010
Epoch 6/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.9143 - loss: 0.2568 - val_accuracy: 0.7300 - val_loss: 0.7446 - learning_rate: 0.0010
Epoch 7/200
98/98 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step - accuracy: 0.9111 - loss: 0.262

### results

Seems overfitted to the training set, with discreancy of the accuracies between the training and the validation set.

In [29]:
y_train_pred = model_Ensemble.predict(train_vectors)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)
y_train_true_classes = np.argmax(train_labels_nn, axis=1)
y_dev_pred = model_Ensemble.predict(dev_vectors)
y_dev_pred_classes = np.argmax(y_dev_pred, axis=1)
y_dev_true_classes = np.argmax(dev_labels_nn, axis=1)

# accu
train_accuracy = accuracy_score(y_train_true_classes, y_train_pred_classes)
dev_accuracy = accuracy_score(y_dev_true_classes, y_dev_pred_classes)

print(f"Train accuracy : {train_accuracy * 100:.2f}%")
print(f"Dev accuracy   : {dev_accuracy * 100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Train accuracy : 92.66%
Dev accuracy   : 77.00%
